# Import Libraries

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import ast
import time
import numpy as np
import csv
import boto3 
import pickle
import os
import matplotlib.pyplot as plt
import requests
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import xgboost
from xgboost import XGBRegressor
from numpy import asarray
from pandas import read_csv
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt 
% matplotlib inline

# Data Processing

In [ ]:
df = pd.read_excel(r'BBC_1000_new.xlsx')
df.shape

In [ ]:
# transform links in column 'SiteLink' to be clickable 
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

df.style.format({'SiteLink': make_clickable})

In [ ]:
# image pre-processing (delete pictures that have no sense with the analysis and check for duplicates in Link Text column)

df1 = df.head(1000)
list1 = df1['SiteLink'].tolist()

l1 = []
for site in list1:
    response = requests.get(site)
    doc = BeautifulSoup(response.text, 'html.parser')
    headlines = doc.find_all('img')
    l1 += [headlines[0]['src']]

counter = -1
link_name = []
for img in l1:
    
    counter = counter + 1    
    # We can split the file based upon / and extract the last split within the python list below:
    file_name = img.split('/')[-1]
    file_name2 = file_name.split('.')[0]
    file_name2 = "_" + str(counter) + "."
    file_name = file_name2 + file_name.split('.')[-1]
    print(f"This is the file name: {file_name}")
    link_name.append(file_name)
    
    # Now let's send a request to the image URL:
    r = requests.get(img, stream=True)
    
    # We can check that the status code is 200 before doing anything else:
    if r.status_code == 200:
        # This command below will allow us to write the data to a file as binary:
        with open(file_name, 'wb') as f:
            for chunk in r:
                f.write(chunk)
    else:
        # We will write all of the images back to the broken_images list:
        broken_images.append(img)

In [ ]:
# create a photo name description to be joined with the main table
imgurl = pd.DataFrame(l1)
imgurl = imgurl.rename(columns={0: 'Link_Name'})
imgurl.index += 1

imgurl2 = pd.DataFrame(link_name)
imgurl2 = imgurl2.rename(columns={0: 'Photo_Name'})
imgurl2.index += 1

imgurl['Photo_Name'] = imgurl2['Photo_Name']
idx = 0
imgurl.insert(idx, 'ImageCode', value=np.arange(len(imgurl)))

df_final = df.head(1000)
df_final = pd.merge(df_final,imgurl[["ImageCode","Photo_Name"]],on='ImageCode')
df_final

# AWS Rekognition Object/Scene Detection API

In [ ]:
########### Paths
# Path to where your want to save the resulting labels
rekog_results_dir = 'C:/Users/sofia/Desktop/ΔΙΠΛΩΜΑΤΙΚΗ/results/'
# Path to where your images are
rekog_images_dir = 'C:/Users/sofia/Desktop/ΔΙΠΛΩΜΑΤΙΚΗ/data/'

In [ ]:
########### Connect to Rekognition API
# Read in your personal keys
personal_access_key = ""
secret_access_key = ""

# Initialize the boto client to access the Rekogniton api
client=boto3.client('rekognition','us-east-1', # or choose the best region for your work, 
                                               # e.g. the location of your S3 bucket if using that method to store images
                    aws_access_key_id = personal_access_key, 
                    aws_secret_access_key = secret_access_key)

In [ ]:
########### Create a list of images to run through the API
# Make a list of all the images in the rekog_data_dir you created
local_images = os.listdir(rekog_images_dir)

In [ ]:
#local_images[:200]
#local_images[200:400]
#local_images[400:600]
#local_images[600:800]
#local_images[800:1000]

In [ ]:
########### Run each image through the API and store the results
##### Detect labels
### Insert 200 pictures each time and change the path of the excel below
holder_labels = []

for imageFile in local_images[800:1000]: # change here each time the number of images to be inserted (200 pictures each time)
    with open(rekog_images_dir + imageFile, 'rb') as image:
            response = client.detect_labels(Image={'Bytes': image.read()})
    
    print('Detected labels for ' + imageFile)
    
    ## If no labels detected, still save the info:
    if len(response['Labels']) == 0:
        print ("No Labels Detected")
        temp_dict = {}
        temp_dict["image_id"] = imageFile
        temp_dict["full_detect_labels_response"] = response
        temp_dict["label_num"] = ''
        temp_dict["label_str"] = ''
        temp_dict["label_conf"] = ''
        holder_labels.append(temp_dict)   
    
    else:
        
        label_counter = 1
        
        for label in response['Labels']:
            print (label['Name'] + ' : ' + str(label['Confidence']))
            temp_dict = {}
            temp_dict["image_id"] = imageFile
            temp_dict["full_detect_labels_response"] = response
            temp_dict["label_num"] = label_counter
            temp_dict["label_str"] = label['Name']
            temp_dict["label_conf"] = label['Confidence']
            label_counter +=1 # update for the next label
            holder_labels.append(temp_dict)
            
len(holder_labels)

In [ ]:
# Write out the results to a csv
with open(rekog_results_dir + 'awsrekognition_detect_labels_5.csv', 'w', newline = '') as csvfile:
    fieldnames = ['image_id', 'full_detect_labels_response',
                  'label_num', 'label_str',
                  'label_conf'
                  ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader() 
    for entry in holder_labels:
        writer.writerow(entry)

In [ ]:
# Create the final dataframe
detect_labels_df1 = pd.read_csv('C:/Users/sofia/Desktop/ΔΙΠΛΩΜΑΤΙΚΗ/results/awsrekognition_detect_labels_1.csv')
detect_labels_df2 = pd.read_csv('C:/Users/sofia/Desktop/ΔΙΠΛΩΜΑΤΙΚΗ/results/awsrekognition_detect_labels_2.csv')
detect_labels_df3 = pd.read_csv('C:/Users/sofia/Desktop/ΔΙΠΛΩΜΑΤΙΚΗ/results/awsrekognition_detect_labels_3.csv')
detect_labels_df4 = pd.read_csv('C:/Users/sofia/Desktop/ΔΙΠΛΩΜΑΤΙΚΗ/results/awsrekognition_detect_labels_4.csv')
detect_labels_df5 = pd.read_csv('C:/Users/sofia/Desktop/ΔΙΠΛΩΜΑΤΙΚΗ/results/awsrekognition_detect_labels_5.csv')
detect_labels_df = pd.concat([detect_labels_df1, detect_labels_df2, detect_labels_df3, detect_labels_df4, detect_labels_df5], ignore_index=True)

df_final.rename(columns = {'Photo_Name':'image_id'}, inplace = True)
df_final = pd.merge(df_final,detect_labels_df,on='image_id',how='right')
df_final.to_excel(r'BBC_final_df.xlsx')
df_final

# Modelling

### Data Pre-processing

In [ ]:
df = pd.read_excel(r'BBC_1000_new_v2.xlsx')

In [ ]:
df['Post_Created_Hour'] = df['Post Created Time'].str[:2]
df['Post_Created_Hour']=pd.to_numeric(df.Post_Created_Hour)
df['Link_Text_Count'] = df['Link Text'].str.len()

In [ ]:
# remove outliers
df.drop(df[df.Likes > 40000].index, inplace=True)

In [ ]:
# remove variables not important for our analysis in this step
df.drop(['Post Created', 'Post Created Date', 'Post Created Time', 'FacebookURL', 'Message', 'SiteLink', 'Link Text', 'ImageCode'], axis=1, inplace=True)
# arrange columns
data = df[['Likes', 'Likes at Posting', 'Followers at Posting', 'Total Interactions', 'Comments', 'Shares', 'Love', 'Wow', 'Haha', 'Sad', 'Angry', 'Care', 'Post_Created_Hour', 'Link_Text_Count', 'Animal', 'Clothing', 'Military', 'Nature', 'Person', 'Vehicle', 'Other', 'Overperforming Score ']]

In [ ]:
print("Number of missing values inside the dataset: ", data.isna().sum().sum())

In [ ]:
data['Likes'].describe()

In [ ]:
data.describe()

In [ ]:
data.hist(grid=False, figsize=(15,12), color='#86bf91', zorder=2, rwidth=0.9)

In [ ]:
#Using Pearson Correlation
import seaborn as sns
plt.figure(figsize=(17,10))
cor = data.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
#Correlation with output variable
# Remove variables correlated with Y and between them > 80% (Remove Total Interactions, Love, Followers at Posting)
cor_target = abs(cor["Likes"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.7]
relevant_features

In [ ]:
# remove variables not important for our analysis and variables correlated > 0.7 with the target variable
data.drop(['Likes at Posting', 'Followers at Posting', 'Total Interactions', 'Comments', 'Shares', 'Love', 'Overperforming Score '], axis=1, inplace=True)
data

### 1st Modeling Part - Regression Analysis

Our purpose is to predict the number of likes of a Facebook post, based on the number of attributes that we have previously observed and the different number of extra features we created after the Image Rekognition implementation using the Clarifai portal. To do that we will use a number of different Regression models, which can be used to predict numerical values, not just classes, in order to rectify which one of these yields the best results for our approach. The steps that we followed were:
 * **`Step_1:`** Split the data into training and test sets
 * **`Step_2:`** Train a number of different regression models (Linear, Random Forest, Decision Tree, XGBoost)
 * **`Step_3:`** Fit the different regression models to the training data
 * **`Step_4:`** Make the necessary predictions
 * **`Step_5:`** Use Grid Search, an Evaluator for our initial predictions and Cross Validation in order to improve our model and find the best model
 * **`Step_6:`** Calculate a number of different metrics (R2, MAE, MSE, RMSE) on test data and print a table showing the basic statistics of the target variable, in order to see if the predictions were worthwhile or not

#### Split dataset into Train/Test sets

In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop('Likes', axis=1)
Y = pd.DataFrame(data.Likes)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
# X_train (70% from all features without the target variable)
# Y_train (70% including only the target variable)
# X_test (30% from all features without the target variable)
# Y_test (30% including only the target variable -> the numbers that we want to see how much close we are)

#### Perform Cross Validation/Grid Search for all models

In [ ]:
# The user can choose one of the below models
a = LinearRegression()
b = RandomForestRegressor(n_estimators = 100, random_state = 0)
c = DecisionTreeRegressor(max_depth=2)

model = b

In [ ]:
# step-1: create a cross-validation scheme
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)

# step-2: specify range of hyperparameters to tune
hyper_params = [{'n_features_to_select': list(range(1, 9))}]

# step-3: perform grid search and specify model
model.fit(X_train, Y_train)
rfe = RFE(model)             

# 3.2 call GridSearchCV()
model_cv = GridSearchCV(estimator = rfe, 
                        param_grid = hyper_params, 
                        scoring= 'r2', 
                        cv = folds, 
                        verbose = 1,
                        return_train_score=True)      

# fit the model
model_cv.fit(X_train, Y_train)  

# cross-validation results
cv_results = pd.DataFrame(model_cv.cv_results_)

# final model (FIX THE n_features_optimal FOR EACH MODEL)
n_features_optimal = 8
model.fit(X_train, Y_train)
rfe = RFE(model, n_features_to_select=n_features_optimal)             
rfe = rfe.fit(X_train, Y_train)

# predict prices of X_test/X_train
Y_pred_test = model.predict(X_test)
Y_pred_train = model.predict(X_train)

# metrics results (na deiksw poia features exoun to antistoixo coeff)
try:
    print('Coefficients:', model.coef_)
    #print('Intercept:', model.intercept_)
    print('Mean absolute error (MAE) Test: %.2f'
          % mean_absolute_error(Y_test, Y_pred_test, sample_weight=None, multioutput='uniform_average')) 
    print('Mean absolute error (MAE) Train: %.2f'
          % mean_absolute_error(Y_train, Y_pred_train, sample_weight=None, multioutput='uniform_average')) 
    print('Mean squared error (MSE) Test: %.2f'
          % mean_squared_error(Y_test, Y_pred_test)) 
    print('Mean squared error (MSE) Train: %.2f'
          % mean_squared_error(Y_train, Y_pred_train)) 
    print('Root mean squared error (RMSE) Test: %.2f'
          % mean_squared_error(Y_test, Y_pred_test, squared=False)) 
    print('Root mean squared error (RMSE) Train: %.2f'
          % mean_squared_error(Y_train, Y_pred_train, squared=False))
    print('Coefficient of determination (R^2) Test: %.2f'
          % r2_score(Y_test, Y_pred_test))
    print('Coefficient of determination (R^2) Train: %.2f'
          % r2_score(Y_train, Y_pred_train))
except:
    print('Coefficients:', model.feature_importances_)
    #print('Intercept:', model.intercept_)
    print('Mean Absolute Error (MAE) Test:', metrics.mean_absolute_error(Y_test, Y_pred_test))
    print('Mean Absolute Error (MAE) Train:', metrics.mean_absolute_error(Y_train, Y_pred_train))
    print('Mean Squared Error (MSE) Test:', metrics.mean_squared_error(Y_test, Y_pred_test))  
    print('Mean Squared Error (MSE) Train:', metrics.mean_squared_error(Y_train, Y_pred_train))  
    print('Root Mean Squared Error (RMSE) Test:', np.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test)))
    print('Root Mean Squared Error (RMSE) Train:', np.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train)))
    print('Coefficient of determination (R^2) Test: %.2f' % r2_score(Y_test, Y_pred_test))
    print('Coefficient of determination (R^2) Train: %.2f' % r2_score(Y_train, Y_pred_train))
    

# plotting cv results
plt.figure(figsize=(16,6))
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_test_score"])
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_train_score"])
plt.xlabel('number of features')
plt.ylabel('r-squared')
plt.title("Optimal Number of Features")
plt.legend(['test score', 'train score'], loc='upper left')


In [ ]:
actual_df = Y_test
actual_df = actual_df.rename(columns={"Likes": "Actual Likes"})
actual_df = actual_df.reset_index(drop=True)
predicted_df = pd.DataFrame(Y_pred_test)
predicted_df = predicted_df.rename(columns={0: "Predicted Likes"})

actual_predicted_df = pd.concat([actual_df, predicted_df], axis=1)

# Comment each time the model that is not used:
# linear_regression_df = actual_predicted_df
# linear_regression_df['Model'] = 'Linear Regression'
# rf_regression_df = actual_predicted_df
# rf_regression_df['Model'] = 'Random Forest Regression'
# dt_regression_df = actual_predicted_df
# dt_regression_df['Model'] = 'Decision Tree Regression'

#### XGBoost Regression

In [ ]:
model = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

model.fit(X_train, Y_train)

Y_pred_test = model.predict(X_test)

In [ ]:
# Evaluating the Algorithm
from sklearn import metrics
print('Coefficients:', model.feature_importances_)
print('Mean Absolute Error (MAE) Test:', metrics.mean_absolute_error(Y_test, Y_pred_test))  
print('Mean Absolute Error (MAE) Train:', metrics.mean_absolute_error(Y_train, Y_pred_train)) 
print('Mean Squared Error (MSE) Test:', metrics.mean_squared_error(Y_test, Y_pred_test))  
print('Mean Absolute Error (MSE) Train:', metrics.mean_squared_error(Y_train, Y_pred_train)) 
print('Root Mean Squared Error (RMSE) Test:', np.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test)))
print('Root Mean Squared Error (RMSE) Train:', np.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train)))
print('Coefficient of determination (R^2) Test: %.2f' % r2_score(Y_test, Y_pred_test))
print('Coefficient of determination (R^2) Train: %.2f' % r2_score(Y_train, Y_pred_train))

# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X_train, Y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

In [ ]:
actual_df = Y_test
actual_df = actual_df.rename(columns={"Likes": "Actual Likes"})
actual_df = actual_df.reset_index(drop=True)
predicted_df = pd.DataFrame(Y_pred_test)
predicted_df = predicted_df.rename(columns={0: "Predicted Likes"})

actual_predicted_df = pd.concat([actual_df, predicted_df], axis=1)
xgb_regression_df = actual_predicted_df
xgb_regression_df['Model'] = 'XGBoost Regression'

#### Final Results

In [ ]:
# Make a box plot from the value ranges (predicted/actual) from all the models (Y_pred - Y_test)

dfs = [linear_regression_df, rf_regression_df, dt_regression_df, xgb_regression_df]
actual_predicted_df2 = pd.concat(dfs)
actual_predicted_df2 = actual_predicted_df2.reset_index(drop=True)
actual_predicted_df2["Actual_Predicted_Differences"] = actual_predicted_df2["Predicted Likes"] - actual_predicted_df2["Actual Likes"]


sns.set_style("whitegrid")
sns.set(rc={'figure.figsize':(12,9)})
sns.boxplot(x = 'Model', y = 'Actual_Predicted_Differences', data = actual_predicted_df2)

In [ ]:
# Make a graph with the predicted vs actual values of the best model
rf_regression_df2 = rf_regression_df.rename(columns={"Predicted Likes": "Predicted_Likes", "Actual Likes": "Actual_Likes"})
Actual_Likes = rf_regression_df2["Actual_Likes"]
Predicted_Likes = rf_regression_df2["Predicted_Likes"]
plt.figure(figsize=(10,10))
plt.scatter(Actual_Likes, Predicted_Likes, c='crimson')
# plt.yscale('log')
# plt.xscale('log')

p1 = max(max(Predicted_Likes), max(Actual_Likes))
p2 = min(min(Predicted_Likes), min(Actual_Likes))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Actual Likes', fontsize=15)
plt.ylabel('Predicted Likes', fontsize=15)
plt.axis('equal')
plt.show()

### 2nd Modeling Part - Classification Analysis

Our purpose is to predict if a Facebook post is succesful or not, based on the number of attributes that we have previously observed and the different number of extra features we created after the Image Rekognition implementation using the Clarifai portal. To do that we will use a number of different Classification models, which can be used to predict classes, in order to rectify which one of these yields the best results for our approach. The steps that we followed were:
 * **`Step_1:`** Create 2 buckets of High/Low number of Likes based on the avg number of Likes
 * **`Step_2:`** Split the data into training and test sets
 * **`Step_3:`** Train a number of different classification models (Logistic Regression, Random Forest Classifier, Decision Tree Classifier)
 * **`Step_4:`** Use Cross Validation in order to improve our models and find the best model
 * **`Step_5:`** Fit the different classification models to the training data
 * **`Step_6:`** Make the necessary predictions
 * **`Step_7:`** Calculate a number of different metrics (Accuracy, Precision, Specificity, Sensitivity, AUC) on test data and print the corresponding classification report of the best model, in order to see if the predictions were worthwhile or not

#### Split Likes on High/Low buckets based on the avg number of Likes

In [ ]:
mean_of_likes = data["Likes"].mean()

def f(row):
    if row['Likes'] > mean_of_likes:
        val = 1
    else:
        val = 0
    return val

data['Likes_Grouped'] = data.apply(f, axis=1)

#### Split dataset into Train/Test sets

In [ ]:
data2 = data.drop('Likes', axis=1)
array = data2.values

# Features: without the target variable
X = array[:,0:-1]

# Target variable: 'Likes_Grouped'
Y = array[:,-1]

# 30% of the data will be used for testing
test_size= 0.30
seed = 7
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, test_size=test_size, random_state=seed)

#### Modeling

In [ ]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('RF', RandomForestClassifier()))
models.append(('DT', DecisionTreeClassifier()))

In [ ]:
results_c = []
names_c = []

for name, model in models:
    # define how to split off validation data ('kfold' how many folds)
    #kfold = KFold(n_splits=10, random_state=seed)    
    kfold = KFold(n_splits=10)
    # train the model
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')    
    results_c.append(cv_results)
    names_c.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

#### Results of Models' Experiments

In [ ]:
# Perfomance of Classification Algorithms Boxplots
fig = plt.figure()
fig.suptitle('Perfomance of Classification Algorithms')
ax = fig.add_subplot(111)
plt.boxplot(results_c)
ax.set_xticklabels(names_c)
plt.show()

In [ ]:
# Accuracy, Sensitivity and Specificity scores of all models

# Fit the models
LR = LogisticRegression()
LR.fit(X_train, Y_train)
RF = RandomForestClassifier()
RF.fit(X_train, Y_train)
DT = DecisionTreeClassifier()
DT.fit(X_train, Y_train)

# Make the predictions
predictions_LR = LR.predict(X_test)
predictions_RF = RF.predict(X_test)
predictions_DT = DT.predict(X_test)

# Accuracy Scores
print("Accuracy Scores")
print("LR:", accuracy_score(Y_test, predictions_LR))
print("RF:", accuracy_score(Y_test, predictions_RF))
print("DT:", accuracy_score(Y_test, predictions_DT))
print("")

# Precision Scores
print("Precision Scores")
print("LR:", precision_score(Y_test, predictions_LR))
print("RF:", precision_score(Y_test, predictions_RF))
print("DT:", precision_score(Y_test, predictions_DT))
print("")

# # Specificity Scores
print("Specificity Scores")
tn_LR, fp_LR, fn_LR, tp_LR = confusion_matrix(Y_test, predictions_LR).ravel()
tn_RF, fp_RF, fn_RF, tp_RF = confusion_matrix(Y_test, predictions_RF).ravel()
tn_DT, fp_DT, fn_DT, tp_DT = confusion_matrix(Y_test, predictions_DT).ravel()
print("LR:", tn_LR / (tn_LR+fp_LR))
print("RF:", tn_RF / (tn_RF+fp_RF))
print("DT:", tn_DT / (tn_DT+fp_DT))
print("")

# Sensitivity Scores
print("Recall Scores")
print("LR:", recall_score(Y_test, predictions_LR))
print("RF:", recall_score(Y_test, predictions_RF))
print("DT:", recall_score(Y_test, predictions_DT))
print("")

### Validation results of the best model

In [ ]:
# Confusion Matrix and Classification Report
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  
conf_mat = confusion_matrix(Y_test, predictions_RF) #confusion matrix
fig, ax = plt.subplots(figsize=(5,3))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=['0', '1'], 
            yticklabels=['0', '1'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion matrix of the Random Forest Classifier')
plt.show()
print(classification_report(Y_test,predictions_RF)) #classification report

In [ ]:
# AUC Curve
from sklearn import metrics
fpr, tpr, _ = metrics.roc_curve(Y_test,  predictions_RF)
auc = metrics.roc_auc_score(Y_test,  predictions_RF)

#create ROC curve
plt.plot(fpr,tpr,label="AUC="+str(auc))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

In [ ]:
# Variable importance of the best model
importances = RF.feature_importances_

import matplotlib.pyplot as plt
 
plt.title('Feature Importance')
plt.bar(range(X_train.shape[1]), importances, align='center')
plt.tight_layout()
plt.show()

In [ ]:
importances = RF.feature_importances_
importances